### Avoided crossing module:

In [ ]:
import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import scipy.constants as sc
import scipy.optimize as fit

import scipy.ndimage as scd

from qkit.analysis import dat_reader as dr
from qkit.analysis.avoided_crossing_fit import avoided_crossing_fit as acf

from qkit.analysis.avoided_crossing_fit.ac_preparation import acp

### Reading Data

In [ ]:
# Switch to measurement directory

os.chdir("")

In [ ]:
# Folder that contains the h5 measurement file

measfolder=""

In [ ]:
# Name of the h5 measurement file

measfile=""

In [ ]:
#Reading the data

data, fn = dr.load_data(measfolder+measfile,['current', 'frequency', 'amplitude'])
curr_raw, freq_raw, signal_raw, signal_time = data

In [ ]:
#Reshape signal (h5 load has shape (x, y, 1), but we need (x, y))

signal_raw = signal_raw.reshape(signal_raw.shape[0], signal_raw.shape[1]).T

In [ ]:
signal = signal_raw
curr = curr_raw
freq = freq_raw

### Detect Crossing

In [ ]:
# Create anticrossing object with ac_preparation by Yannick Schön

ac=acp(signal,curr,freq)

In [ ]:
# Search the peaks corresponding to the two arms of the anticrossing

# f0_0: Frequency of the resonator
# df: Span around f0 to be searched in
# f0_e: Frequency of the resonator after the crossing
# thres: Threshold parameter for peakutils (standard: 0.3)
# min_dist: Minimum distance between identified peaks (standard: 30)

ac.peaks(f0_0=5.0e9,df=6.0e6,min_dist=50)

In [ ]:
# Plot the data and the found peaks

ac.plot_peaks()

In [ ]:
# If needed cut # points from end of arm 1

#ac.cut1(5)

In [ ]:
# If needed cut # points from end of arm 2

#ac.cut2(5)

### Fit

In [ ]:
# Use avoided_crossing_fit by Alex Stehli to fit the detected anticrossing

fig, axes = plt.subplots(figsize=(16,8))

fit_params, fit_errors = acf.crossing_fit(ac.fitcurrs1, ac.fitcurrs2, ac.fitfreqs1/1e9, ac.fitfreqs2/1e9)

In [ ]:
# Optional temperature information to be shown in legend

temp=None

In [ ]:
# Create a nice overlay of data and fit:

# Create contens of fit parameter label
fitparams_label="w1 = ( {0:7.3f} +- {1:8.3f}) GHz".format(fit_params[0], (np.diag(fit_errors)[0])**0.5)
fitparams_label+="\nw2 = ( {0:7.3f} +- {1:8.3f}) GHz".format(fit_params[1], (np.diag(fit_errors)[1])**0.5)
fitparams_label+="\na  = ( {0:7.3f} +- {1:8.3f}) GHz/[x]".format(fit_params[2], (np.diag(fit_errors)[2])**0.5)
fitparams_label+="\ng  = ( {0:7.3f} +- {1:8.3f}) MHz".format(fit_params[3]*1000, 1000*(np.diag(fit_errors)[3])**0.5)
if temp: fitparams_label+="\n\nTemperature: "+temp

# Shift current for pcolormesh (looks a lot better)
fig, axes = plt.subplots(figsize=(18,12))

cp = curr - .5*(curr[1]- curr[0])*np.ones(curr.shape)
plt.pcolormesh(cp, freq*10**-9, signal, cmap='coolwarm')
cbar = plt.colorbar()
xlin = np.linspace(cp.min(), cp.max(), 100)
plt.plot(xlin, acf.avoided_crossing(xlin, fit_params, -1), 'r')
plt.plot(xlin, acf.avoided_crossing(xlin, fit_params, 1), 'r', label="Fit parameters:\n"+fitparams_label)
plt.xlim(cp.min(), cp.max())
plt.ylim(freq.min()*10**-9, freq.max()*10**-9)
plt.xlabel(r'$I_{coil}^{dc}\,(\mathrm{mA})$', fontsize=27)
plt.ylabel(r'$f\,(\mathrm{GHz})$', fontsize=27)
cbar.set_label(r'$\phi\,(\mathrm{a.u.})$', rotation='90', fontsize = 27, labelpad = 15)
plt.legend(loc="best")

# Save the data if wished

#plt.savefig(measfolder+measfile+".png", dpi = 600)
#np.savetxt(measfolder+"fitparameters.txt",fit_params)